# Fill test data glosses/examples with LLMs

In [1]:
import pandas as pd
import json
import os
import openai
from dotenv import load_dotenv
import time
import re
import unicodedata
import numpy as np
pd.options.mode.chained_assignment = None  # default='warn'

## 1. Load the data and compute the number of requests to make

In [2]:
FILE_TO_READ = './data/augmented/axolotl.dev.ru.tsv'
RESPONSES_FILE_1 = './data/augmented/llm-responses/responses-ru-dev-1.json'
RESPONSES_FILE_2 = './data/augmented/llm-responses/responses-ru-dev-2.json'
AUGMENTED_EXAMPLES = 3

language = FILE_TO_READ.split('.')[-2]
filename = FILE_TO_READ.split('/')[-1].split('.')[0:-1]
filename = '.'.join(filename)
language, filename

('ru', 'axolotl.dev.ru')

The following requets will be done:
1. For each glos-known row that doesn't have an example available, an example will be generated (+3)**
2. For each gloss-unknown row, a gloss is generated. This is also useful for achieving directly subtask 2 (+1)
3. For each gloss-known row, at least three examples are needed, so if a sense has only one example, two more are generated (+2)

** To avoid repeating requests, the generation of step 3. is also done in step 1. directly for non-example rows.

In [3]:
df = pd.read_csv(FILE_TO_READ, sep='\t')
df.info()
print("\nColumns to fill 'example':" , df['example'].isna().sum(), "(output ×3)")
print("Columns to fill 'gloss':" , df['gloss'].isna().sum(), "(output ×1)")
print("Columns to augment:" , df[~df['gloss'].isna()].shape[0]-df['example'].isna().sum(), "(output ×2)")
print("Total:", df['example'].isna().sum() + df['gloss'].isna().sum() + df[~df['gloss'].isna()].shape[0], "requests")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2026 entries, 0 to 2025
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   usage_id              2026 non-null   object 
 1   word                  2026 non-null   object 
 2   orth                  2026 non-null   object 
 3   sense_id              421 non-null    object 
 4   gloss                 421 non-null    object 
 5   example               1912 non-null   object 
 6   indices_target_token  0 non-null      float64
 7   date                  2026 non-null   object 
 8   period                2026 non-null   object 
dtypes: float64(1), object(8)
memory usage: 142.6+ KB

Columns to fill 'example': 114 (output ×3)
Columns to fill 'gloss': 1605 (output ×1)
Columns to augment: 307 (output ×2)
Total: 2140 requests


## 2. Define methods to send and store requests to OpenAI's API

In [4]:
load_dotenv('./gpt35.env')
openai.api_type = "azure"
openai.api_version = os.getenv("OPENAI_API_VERSION")
api_key = os.getenv("OPENAI_API_KEY")
api_base = os.getenv("OPENAI_ENDPOINT")
engine=os.getenv("OPENAI_IMPLEMENTATION")
model=os.getenv("MODEL_NAME")
model

'gpt-3.5-turbo'

In [5]:
MAX_ITERATIONS = 1 # only one repetition is allowed
def request(prompt, max_tokens=500, temperature=0, iterations=0):
    """Request a completion from the OpenAI API.
    :param prompt: The prompt to send to the API
    :param max_tokens: The maximum number of tokens in the output
    :param temperature: The degree of randomness in the output
    :return response: The response from the API
    :return usage: The count of token usage from the API { "input", "output" }
    """
    try:
        response = openai.ChatCompletion.create(
            api_key=api_key,
            api_base=api_base,
            engine=engine,
            max_tokens=max_tokens,
            temperature=temperature,
            messages=[{"role": "user", "content": prompt}],
        )
        finish_reason = response["choices"][0]["finish_reason"]
        rsp = "" if finish_reason == "content_filter" else response["choices"][0]["message"]["content"]
        return rsp, { "input": response["usage"]["prompt_tokens"], "output": response["usage"]["completion_tokens"] }, finish_reason
    except Exception as e:
        response = ""
        usage = {"input": 0, "output": 0}
        if f"{e}".startswith("The response was filtered due to the prompt triggering Azure OpenAI's content management policy"):
            finish_reason = "content_filter"
        else:
            if ("Max retries exceeded with url" in f"{e}") and (iterations < MAX_ITERATIONS):
                    print("MAX_RETRIES triggered. Retrying in 60 seconds...")
                    time.sleep(60)
                    return request(prompt, max_tokens, temperature, iterations+1) # retry after 60 seconds
            finish_reason = f"ERROR [{type(e).__name__}]: {e}"
        return response, usage, finish_reason

gen_prompt = lambda prompt, *args: prompt.format(*args)

Finally, a method to find the target word within the sentence (wether the "word" or the "orth"):

In [6]:
def generate_substrings(word):
    substrings = []
    for i in range(len(word), 0, -1):
        substrings.append(word[:i])
    return substrings[1:-1]

def find_sub_list(sl,l): # not used because some examples have no exact coincidence
    sll=len(sl)
    for ind in (i for i,e in enumerate(l) if e==sl[0]):
        if l[ind:ind+sll]==sl:
            return ind+1,ind+sll # +1 for the [CLS] token

def extract_letters(input_string):
    return re.sub(r'[^а-яА-Яa-zA-ZÀ-ÿёЁ\u0300-\u036f-]', '', input_string)

def remove_accents(word):
    normalized_word = unicodedata.normalize('NFD', word)
    cleaned_word = re.sub(r'[\u0300-\u036f]', '', normalized_word)
    return cleaned_word

def find_word_containing_target(sentence, target_word):
    index = sentence.find(target_word)
    if index == -1:
        return None
    start_index = sentence.rfind(" ", 0, index) + 1 if index != 0 else 0
    end_index = sentence.find(" ", index + len(target_word)) if sentence.find(" ", index + len(target_word)) != -1 else len(sentence)
    final_char = final_char = " " if target_word.endswith(" ") else ""
    return sentence[start_index:end_index].split()[0] + final_char

def get_search(example, word, orth=None, print_search=False):
    # append the words to search in the example, in the desired ORDER
    # 1 - the word (with an ending character), and the word itself
    search = [f"{word} ", f"{word},", f"{word}.", word]

    # 2 - the orthographic form (with an ending character), and the orthographic form itself
    if orth and orth != word:
        search += [f"{orth} ", f"{orth},", f"{orth}.", orth]

    # 3 - all substrings of the word (i.e. выходить -> ['выходит', 'выходи', 'выход', 'выхо', 'вых', 'вы'])
    search += generate_substrings(word)

    # 4 - all substrings of the orthographic form
    if orth:
        search.extend([i for i in generate_substrings(orth) if i not in search])

    # 5 - the word without accents (with an ending character), and the word without accents itself
    unicoded_word = remove_accents(word)
    if unicoded_word != word:
        search += [f"{unicoded_word} ", f"{unicoded_word},", f"{unicoded_word}.", unicoded_word]
    
    # 6 - the orthographic form without accents (with an ending character), and the orthographic form without accents itself
    if orth and orth != word:
        unicoded_orth = remove_accents(orth)
        if unicoded_orth != orth:
            search += [f"{unicoded_orth} ", f"{unicoded_orth},", f"{unicoded_orth}.", unicoded_orth]

    # 7 - all substrings of the word without accents
    if unicoded_word != word:
        search.extend([i for i in generate_substrings(unicoded_word) if i not in search])

    # 8 - all substrings of the orthographic form without accents
    if orth and orth != word and unicoded_orth != orth:
        search.extend([i for i in generate_substrings(unicoded_orth) if i not in search])

    if print_search:
        print(f"Searching for: {search}")

    # FIND the first search-string that is within the example, if any (in upper or lowercase)
    for s in search:
        search_word = find_word_containing_target(example, s)
        if search_word:
            break
        search_word = find_word_containing_target(example.lower(), s.lower())
        if search_word:
            index = example.lower().find(search_word)
            if index == -1:
                # this should never happen
                raise Exception(f"Found '{search_word}' in '{example.lower()}', but then not found...")
            else:
                search_word = example[index:index + len(search_word)]
            break
    else:
        search_word = ""
    return extract_letters(search_word)

In [7]:
print(f'Find {(df.loc[0, "word"], df.loc[0, "orth"])} in the sentence: {df.loc[0, "example"]}')
result_example = get_search(df.loc[0, "example"], df.loc[0, "word"], df.loc[0, "orth"], print_search=True)
print(f"Found: {result_example}" if result_example else "Not found")

Find ('могильник', 'могильникъ') in the sentence: могилки, кладбище
Searching for: ['могильник ', 'могильник,', 'могильник.', 'могильник', 'могильникъ ', 'могильникъ,', 'могильникъ.', 'могильникъ', 'могильни', 'могильн', 'могиль', 'могил', 'моги', 'мог', 'мо']
Found: могилки


## 3. Prepare and send requests for each of the 3 cases (2 in the same)

### 1. For each glos-known row that doesn't have an example available, an example will be generated (+1)
&&
### 3. For each gloss-known row, two more examples will be generated (data augmentation) (+2)
Additionally, in this step, the data is augmented: an extra-2 examples are generated for each request (3 in total)

#### 1.1. Prepare dataset and prompt

In [8]:
df[~df['gloss'].isna()][['word', 'orth', 'sense_id', 'gloss', 'example']]

,word,orth,sense_id,gloss,example
0,могильник,могильникъ,mogil'nik_UYTE5-B076I,вят. слово из Вятской губернии арх. слово из А...,"могилки, кладбище"
1,могильник,могильникъ,mogil'nik_C3GhETZc5Vs,ярс. слово из Ярославской губернии крупный коч...,NaN
2,могильник,могильникъ,mogil'nik_KNs3eVn3pFY,арх. слово из Архангельской губернии походный ...,NaN
4,могильник,могильникъ,mogil'nik_ahboIs9hMMk,орнитол. хищная птица рода крупных степных орлов,Орел могильник
5,могильник,могильникъ,mogil'nik_2g5paR8fF5I,"могильщик, могиляк, могилокопатель гробокоп, р...","Это я, я, который в """"Гамлете"""" могильщика игр..."
...,...,...,...,...,...
1979,говно,говно,govno_Ku2LXYd922U,"дермо, кал, помет, испражнение, экскремент.",Свое говно не воняет. С говна пенку снимать.
1985,голубой,голубой,goluboj_vduSkjU0I6A,"имеющий светло-синий цвет, цве́та чистого безо...",Не подстать синему кафтану голубой подбой. Нар...
1995,горбатый,горбатый,gorbatyj_7iVqG7pI9R8,"разг. тот, у кого есть горб","От работы не будешь богат, а будешь горбат. Му..."
2016,горячка,горячка,gorjachka_2yFyKaEbJEk,"разг. о человеке горячего, вспыльчивого нрава","Человек горячка, запальчивый. Пороть горячку,"


In [9]:
# original (english): 'Give me ONLY {} line-separated usage examples of the word "{}" with the following definition:\n{}'

if language == "ru":
    prompt = 'Приведите мне ТОЛЬКО {} примера употребления слова "{}" в строках со следующим определением:\n{}'
elif language == "fi":
    prompt = 'Anna minulle VAIN {} rivieroteltua esimerkkiä sanasta "{}" seuraavalla määritelmällä:\n{}'
else: # language == "de"
    prompt = 'Geben Sie mir NUR {} durch Zeilen getrennte Verwendungsbeispiele für das Wort „{}“ mit der folgenden Definition:\n{}'

exp_fill = df[~df['gloss'].isna()]
for i in range(5):
    print(prompt.format(AUGMENTED_EXAMPLES, exp_fill.iloc[i, 1], exp_fill.iloc[i, 4]), end="\n\n")
del exp_fill

Приведите мне ТОЛЬКО 3 примера употребления слова "могильник" в строках со следующим определением:
вят. слово из Вятской губернии арх. слово из Архангельской губернии кур.? могила, в знач. значение насыпи над покойником, дерновая кладка, памятник

Приведите мне ТОЛЬКО 3 примера употребления слова "могильник" в строках со следующим определением:
ярс. слово из Ярославской губернии крупный кочкарник, кочегурник;

Приведите мне ТОЛЬКО 3 примера употребления слова "могильник" в строках со следующим определением:
арх. слово из Архангельской губернии походный чемоданчик с иглами, шильями, нитками, дратвой ипр. для чинки одежи и обуви.

Приведите мне ТОЛЬКО 3 примера употребления слова "могильник" в строках со следующим определением:
орнитол. хищная птица рода крупных степных орлов

Приведите мне ТОЛЬКО 3 примера употребления слова "могильник" в строках со следующим определением:
могильщик, могиляк, могилокопатель гробокоп, рабочий при кладбище.



In [10]:
r = {"data":[], "checkpoint": 0, "input_tokens": 0, "output_tokens": 0}
if os.path.exists(RESPONSES_FILE_1):
    with open(RESPONSES_FILE_1, "r") as f:
        r = json.load(f)
else:
    with open(RESPONSES_FILE_1, "w") as f:
        f.write(json.dumps(r, indent=4))

#### 1.2. Send requests

In [11]:
def generate_examples(n, prompt):
    response, usage, finish_reason = request(prompt)
    print(response)

    status = "ok"
    lines = response.split('\n')
    if finish_reason != "stop":
        status = "error"
        new_lines = [f"Error {finish_reason}" for _ in range(n)]
    else:
        if n == 1:
            if len(lines) > 1:
                if len(lines) > 2:
                    print(f"WARNING 1: {len(lines)} lines found, but {n} expected.")
                    status = "warning"
                new_lines = [lines[-1]]
            new_lines = lines
        else:
            new_lines = []
            for line in lines:
                if (line != "") and ((line[0].isdigit()) or line[0] == '-'):
                    line = ' '.join(line.split(' ')[1:])
                    new_lines.append(line)
            if len(new_lines) != n:
                print(f"WARNING 2: {len(new_lines)} lines found, but {n} expected.")
                status = "warning"

    r['input_tokens'] += usage['input']
    r['output_tokens'] += usage['output']
    r['data'].append({"prompt":prompt, "response":response, "finish_reason":finish_reason, "status":status})
    with open(RESPONSES_FILE_1, "w", encoding='utf-8') as f:
        f.write(json.dumps(r, indent=4))

    return new_lines

In [17]:
df = pd.read_csv(FILE_TO_READ, sep='\t')
mod_df = df.copy()
id_template = '_'.join(df.loc[0, 'usage_id'].split("_")[:2])
last_id = int(df.loc[len(df)-1, 'usage_id'].split("_")[-1])

index = 0

In [18]:
while index < len(df):
    row = mod_df.iloc[index]
    gloss = row['gloss']
    if f"{gloss}" != "nan":
        sense_id = row['sense_id']
        word = row['word']
        example = row['example']
        print(f"---------------------------------------- {index, sense_id} ----------------------------------------")

        found = len(mod_df[(mod_df['sense_id'] == sense_id) & (~mod_df['example'].isna())])
        n = AUGMENTED_EXAMPLES - found
        if n > 0:
            p = gen_prompt(prompt, n, word, gloss)
            print(p, end="\n\n")
            examples = generate_examples(n, p)
            if f"{example}" == "nan":
                row['example'] = examples[0]
                mod_df.iloc[index] = row
                start_at = 1
            else:
                start_at = 0
            for e in examples[start_at:]:
                last_id += 1
                row['example'] = e
                row['usage_id'] = f"augmented_{id_template}_{last_id}"
                mod_df.loc[len(mod_df)] = row
        else:
            print(f"No examples needed for {word} ({sense_id}), found {found}")
    index += 1

mod_df.to_csv(f'./data/augmented/{filename}.tsv', sep='\t', index=False)
mod_df

---------------------------------------- (0, "mogil'nik_UYTE5-B076I") ----------------------------------------
No examples needed for могильник (mogil'nik_UYTE5-B076I), found 3
---------------------------------------- (1, "mogil'nik_C3GhETZc5Vs") ----------------------------------------
No examples needed for могильник (mogil'nik_C3GhETZc5Vs), found 3
---------------------------------------- (2, "mogil'nik_KNs3eVn3pFY") ----------------------------------------
No examples needed for могильник (mogil'nik_KNs3eVn3pFY), found 3
---------------------------------------- (4, "mogil'nik_ahboIs9hMMk") ----------------------------------------
No examples needed for могильник (mogil'nik_ahboIs9hMMk), found 3
---------------------------------------- (5, "mogil'nik_2g5paR8fF5I") ----------------------------------------
No examples needed for могильник (mogil'nik_2g5paR8fF5I), found 3
---------------------------------------- (6, "mogil'nik_Qwr4RrN_r4U") ----------------------------------------
No e

,usage_id,word,orth,sense_id,gloss,example,indices_target_token,date,period
0,dev_ru_0,могильник,могильникъ,mogil'nik_UYTE5-B076I,вят. слово из Вятской губернии арх. слово из А...,"могилки, кладбище",NaN,old,old
1,dev_ru_1,могильник,могильникъ,mogil'nik_C3GhETZc5Vs,ярс. слово из Ярославской губернии крупный коч...,В окрестностях Ярославля можно встретить много...,NaN,old,old
2,dev_ru_2,могильник,могильникъ,mogil'nik_KNs3eVn3pFY,арх. слово из Архангельской губернии походный ...,В музее была выставлена коллекция архангельски...,NaN,old,old
3,dev_ru_3,могильник,могильникъ,NaN,NaN,"Орёл изображает реку Халзан, его голова ― скал...",NaN,new,new
4,dev_ru_4,могильник,могильникъ,mogil'nik_ahboIs9hMMk,орнитол. хищная птица рода крупных степных орлов,Орел могильник,NaN,old,old
...,...,...,...,...,...,...,...,...,...
2857,augmented_dev_ru_3292,горбатый,горбатый,gorbatyj_7iVqG7pI9R8,"разг. тот, у кого есть горб","В книге описывается горбатый старый ведьмак, к...",NaN,old,old
2858,augmented_dev_ru_3293,горячка,горячка,gorjachka_2yFyKaEbJEk,"разг. о человеке горячего, вспыльчивого нрава","""Он всегда был горячкой и не мог сдержать свои...",NaN,old,old
2859,augmented_dev_ru_3294,горячка,горячка,gorjachka_2yFyKaEbJEk,"разг. о человеке горячего, вспыльчивого нрава","""Его горячка иногда приводила к непредсказуемы...",NaN,old,old
2860,augmented_dev_ru_3295,горячка,горячка,gorjachka_uFKY5ZkBVB0,"устар. прост. болезнь, сопровождаемая высокой ...","""У него началась горячка, и он сразу же лег в ...",NaN,old,old


In [20]:
for i in r['data']:
    if i['status'] == 'error':
        print("-------------------------------------------")
        print(i['prompt'])
        print()
        print(i["response"])

-------------------------------------------
Приведите мне ТОЛЬКО 2 примера употребления слова "лизать" в строках со следующим определением:
разг., фам., неодобр., кого-что целовать


-------------------------------------------
Приведите мне ТОЛЬКО 3 примера употребления слова "парафраз" в строках со следующим определением:
филос. замещение, вытеснение


-------------------------------------------
Приведите мне ТОЛЬКО 2 примера употребления слова "перевёртыш" в строках со следующим определением:
дать кому тычка, затрещину, зашеину.


-------------------------------------------
Приведите мне ТОЛЬКО 2 примера употребления слова "пронзить" в строках со следующим определением:
проколов, проткнув, нанести кому-либо сквозную рану


-------------------------------------------
Приведите мне ТОЛЬКО 2 примера употребления слова "прус" в строках со следующим определением:
город Пруса


-------------------------------------------
Приведите мне ТОЛЬКО 2 примера употребления слова "распускать" в стро

### 2. For each gloss-unknown row, a gloss is generated. This is also useful for achieving directly subtask 2 (+1)

#### 2.1. Prepare dataset and prompt

In [21]:
df[df['gloss'].isna()][['word', 'orth', 'sense_id', 'gloss', 'example']]

,word,orth,sense_id,gloss,example
3,могильник,могильникъ,NaN,NaN,"Орёл изображает реку Халзан, его голова ― скал..."
7,могильник,могильникъ,NaN,NaN,В экспозиции представлен инвентарь из реликтов...
8,могильник,могильникъ,NaN,NaN,Близ Барбашиной поляны отрыт мордовский могиль...
9,могильник,могильникъ,NaN,NaN,О строительстве могильника для захоронения рад...
10,могильник,могильникъ,NaN,NaN,"Якобы тут, возле Черниховки, находится радиоак..."
...,...,...,...,...,...
2021,горячка,горячка,NaN,NaN,Надо было опять торопиться. Но недавняя горячк...
2022,горячка,горячка,NaN,NaN,"И вторая нога вошла в чулок, вернее, чулок поп..."
2023,горячка,горячка,NaN,NaN,В порыве своей первой творческой горячки он во...
2024,горячка,горячка,NaN,NaN,В суматохе и горячке последних часов кого-нибу...


In [22]:
# original (english): 'What does "{}" mean in the following examples (ONLY return JSON with the key "definition" - detailed definition)?\n{}'

if language == "ru":
    prompt = 'Что означает «{}» в следующих примерах (ТОЛЬКО возвращайте JSON с ключом «определение» — подробное определение)?\n{}'
elif language == "fi":
    prompt = 'Mitä «{}» tarkoittaa seuraavissa esimerkeissä (palauta VAIN JSON avaimella "definition" - monisanainen määritelmä)?\n{}'
else: # language == "de"
    prompt = 'Was bedeutet „{}“ in den folgenden Beispielen (geben sie JSON NUR mit dem Schlüssel „definition“ zurück – ausführliche Definition)?\n{}'

gls_fill = df[df['gloss'].isna()]
for i in range(5):
    print(prompt.format(gls_fill.iloc[i, 1], gls_fill.iloc[i, 5]), end="\n\n")
del gls_fill

Что означает «могильник» в следующих примерах (ТОЛЬКО возвращайте JSON с ключом «определение» — подробное определение)?
Орёл изображает реку Халзан, его голова ― скалу на Халзане, а лапы могильника ― могилу на скале.

Что означает «могильник» в следующих примерах (ТОЛЬКО возвращайте JSON с ключом «определение» — подробное определение)?
В экспозиции представлен инвентарь из реликтового языческого балтского женского захоронения в курганном могильнике конца XIII ― начала XIV века.

Что означает «могильник» в следующих примерах (ТОЛЬКО возвращайте JSON с ключом «определение» — подробное определение)?
Близ Барбашиной поляны отрыт мордовский могильник XIV века, и в нем найдено много ценных в археологическом отношении предметов.

Что означает «могильник» в следующих примерах (ТОЛЬКО возвращайте JSON с ключом «определение» — подробное определение)?
О строительстве могильника для захоронения радиационных отходов с отработавших подводных лодок мы доложим дополнительно, когда проработаем вопрос о

In [23]:
r2 = {"data":[], "checkpoint": 0, "input_tokens": 0, "output_tokens": 0}
if os.path.exists(RESPONSES_FILE_2):
    with open(RESPONSES_FILE_2, "r") as f:
        r2 = json.load(f)
else:
    with open(RESPONSES_FILE_2, "w") as f:
        f.write(json.dumps(r2, indent=4))

#### 2.2. Send requests

In [24]:
def generate_gloss(prompt):
    response, usage, finish_reason = request(prompt)
    print(response)

    status = "ok"

    if finish_reason != "stop":
        status = "error"
        gloss = f"Error {finish_reason}"
    else:
        json_match = re.search(r'{.*}', response, re.DOTALL)
        if json_match:
            json_string = json_match.group(0)
            try:
                glosses = list(json.loads(json_string).values())
            except json.JSONDecodeError as e:
                if language == "ru":
                    pattern = r'(?<="определение": ").*(?=")'
                else:
                    pattern = r'(?<="definition": ").*(?=")'
                definition = re.search(pattern, json_string).group()
                if definition:
                    glosses = ['"' + definition]
                else:
                    raise e
        else:
            print("No JSON found in the input string.")
            status = "ok_nojson"
            lines = response.split('\n')
            if len(lines) > 1:
                print(f"WARNING 1: {len(lines)} lines found, but 1 expected.")
                status = "warning"
                glosses = [lines[-1]]
            else:
                glosses = [response]
        if len(glosses) > 1:
            print(f"WARNING 2: {len(glosses)} glosses found, but 1 expected.")
            status = "warning"
        gloss = glosses[-1]
    
    r2['input_tokens'] += usage['input']
    r2['output_tokens'] += usage['output']
    r2['data'].append({"prompt":prompt, "response":response, "finish_reason":finish_reason, "status":status})
    with open(RESPONSES_FILE_2, "w", encoding='utf-8') as f:
        f.write(json.dumps(r2, indent=4))

    if language == "ru":
        gloss = gloss.replace("в данном контексте означает", "-")
    # fi : tarkoittaa
    # de : In diesem Kontext bedeutet | bedeutet in diesem Kontext,
    return gloss

In [25]:
mod_df = pd.read_csv(f'./data/augmented/{filename}.tsv', sep='\t')
df_2 = mod_df.copy()
mod_df_2 = df_2.copy()

for index, row in mod_df_2.iterrows():
    gloss = row["gloss"]
    if f"{gloss}" == "nan":
        example = row["example"]
        word = row["word"]
        orth = row["orth"]
        s = get_search(example, word, orth)
        print(f"---------------------------------------- {index, s} ----------------------------------------")
        if s == "":
            s = word
        p = gen_prompt(prompt, s, example)
        print(p, end="\n\n")
        try:
            gloss = generate_gloss(p)
        except json.JSONDecodeError:
            print("DECODING ERROR - TERMINATING")
            break
        row['gloss'] = gloss
        mod_df_2.iloc[index] = row

mod_df_2.to_csv(f'./data/augmented/{filename}.tsv', sep='\t', index=False)
mod_df_2

---------------------------------------- (3, 'могильника') ----------------------------------------
Что означает «могильника» в следующих примерах (ТОЛЬКО возвращайте JSON с ключом «определение» — подробное определение)?
Орёл изображает реку Халзан, его голова ― скалу на Халзане, а лапы могильника ― могилу на скале.

{
  "определение": "Могильник - это место, где находятся могилы или гробницы, предназначенные для захоронения умерших."
}
---------------------------------------- (7, 'могильнике') ----------------------------------------
Что означает «могильнике» в следующих примерах (ТОЛЬКО возвращайте JSON с ключом «определение» — подробное определение)?
В экспозиции представлен инвентарь из реликтового языческого балтского женского захоронения в курганном могильнике конца XIII ― начала XIV века.

{
  "определение": "Могильник - это место, где находятся могилы или захоронения. В данном контексте, могильник представляет собой место, где расположены захоронения женщины, принадлежащей к ре

,usage_id,word,orth,sense_id,gloss,example,indices_target_token,date,period
0,dev_ru_0,могильник,могильникъ,mogil'nik_UYTE5-B076I,вят. слово из Вятской губернии арх. слово из А...,"могилки, кладбище",NaN,old,old
1,dev_ru_1,могильник,могильникъ,mogil'nik_C3GhETZc5Vs,ярс. слово из Ярославской губернии крупный коч...,В окрестностях Ярославля можно встретить много...,NaN,old,old
2,dev_ru_2,могильник,могильникъ,mogil'nik_KNs3eVn3pFY,арх. слово из Архангельской губернии походный ...,В музее была выставлена коллекция архангельски...,NaN,old,old
3,dev_ru_3,могильник,могильникъ,NaN,"Могильник - это место, где находятся могилы ил...","Орёл изображает реку Халзан, его голова ― скал...",NaN,new,new
4,dev_ru_4,могильник,могильникъ,mogil'nik_ahboIs9hMMk,орнитол. хищная птица рода крупных степных орлов,Орел могильник,NaN,old,old
...,...,...,...,...,...,...,...,...,...
2857,augmented_dev_ru_3292,горбатый,горбатый,gorbatyj_7iVqG7pI9R8,"разг. тот, у кого есть горб","В книге описывается горбатый старый ведьмак, к...",NaN,old,old
2858,augmented_dev_ru_3293,горячка,горячка,gorjachka_2yFyKaEbJEk,"разг. о человеке горячего, вспыльчивого нрава","""Он всегда был горячкой и не мог сдержать свои...",NaN,old,old
2859,augmented_dev_ru_3294,горячка,горячка,gorjachka_2yFyKaEbJEk,"разг. о человеке горячего, вспыльчивого нрава","""Его горячка иногда приводила к непредсказуемы...",NaN,old,old
2860,augmented_dev_ru_3295,горячка,горячка,gorjachka_uFKY5ZkBVB0,"устар. прост. болезнь, сопровождаемая высокой ...","""У него началась горячка, и он сразу же лег в ...",NaN,old,old


In [27]:
for i in r2['data']:
    if i['status'] == 'error':
        print("-------------------------------------------")
        print(i['prompt'])
        print()
        print(i["response"])

-------------------------------------------
Что означает «отрубил» в следующих примерах (ТОЛЬКО возвращайте JSON с ключом «определение» — подробное определение)?
В модели есть настроение и виден замысел: Персей только что отрубил голову Медузы, схватил её за волосы и высоко поднял левой рукой, а сам задумчиво смотрит, наклонив голову, как кровь густым потоком льется из перерубленной шеи.


-------------------------------------------
Что означает «пронизать» в следующих примерах (ТОЛЬКО возвращайте JSON с ключом «определение» — подробное определение)?
Я пробирался туда, где гуще зелень, где смолистее запах деревьев и куда веселее заглядывал солнечный луч, радуясь, что удалось там и сям пронизать мглистую густоту листьев.

{
  "определение": "проникать, проникнуть, проникнуться, проникнуться, проникнуться, проникнуться, проникнуться, проникнуться, проникнуться, проникнуться, проникнуться, проникнуться, проникнуться, проникнуться, проникнуться, проникнуться, проникнуться, проникнуться, пр